In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from collections import deque
import random
import math
import time
import networkx 
from tqdm import tqdm

# RQ1

In [2]:
df = pd.read_csv('wiki-topcats-reduced.txt', sep=r'\t', header=None,  names=['Source','Destination'])

In [3]:
df.head()

,Source,Destination
0,52,401135
1,52,1069112
2,52,1163551
3,62,12162
4,62,167659


__Number of Edges__

In [4]:
num_of_edges = df.shape[0]
print('Number of Edges:', num_of_edges)

Number of Edges: 2645247


__Number of Nodes__

In [5]:
nodes = set(df['Source'].values).union(set(df['Destination'].values))
num_of_nodes = len(nodes)
print('Number of Nodes:', num_of_nodes)

Number of Nodes: 461193


__Checking to see if our graph is directed?__ <br>

we create the graph once for source nodes as the keys and once for destination nodes as our keys. If intersection set between a similar key in two dictionaries is not empty, we have a proof to show our graph is directed. 

In [6]:
# dictionary with source nodes as the key
out_Link = df.groupby(['Source'])['Destination'].apply(list).to_dict()

In [7]:
# dictionary with destination nodes as the key
in_Link = df.groupby(['Destination'])['Source'].apply(list).to_dict()

In [8]:
# checking how many similar keys have non empty intersection sets?
intersect_ = {}
intersect = 0
for item in out_Link.keys():
    if item in in_Link.keys():
        intersect = set(out_Link[item]).intersection(set(in_Link[item]))
        if len(intersect)>0:   
            intersect_[item] = set(out_Link[item]).intersection(set(in_Link[item]))
print(len(intersect_))

249369


__We have enough proof to show that our graph is Directed!__<br><br>


__The average node degree__

In [9]:
average_degree = num_of_edges/num_of_nodes
print('average node degree:', average_degree)

average node degree: 5


__Checking density of our graph__

In [10]:
D = num_of_edges/(num_of_nodes*(num_of_nodes-1))
print('graph density:', D)

graph density: 1.2436602635647606e-05


If D is close to 1 it means that graph is dense, so our graph is not dense (close to 0).

# RQ2


__Categories data set__

In [11]:
categories = {}
with open('wiki-topcats-categories.txt') as file:
    for list_ in file:
        if 20000>len(list_.split()[1:])>3500: #we choose only those categories with number of nodes bigger that 3500
            categories[list_.split()[0][9:-1]] = list(map(int,list_.split()[1:])) #a dictionary with 'key' as category name and values a list of integers, each integer indicates a node

In [12]:
categories.keys()

dict_keys(['English_footballers', 'The_Football_League_players', 'Association_football_forwards', 'Association_football_goalkeepers', 'Association_football_midfielders', 'Association_football_defenders', 'Year_of_birth_unknown', 'Harvard_University_alumni', 'Major_League_Baseball_pitchers', 'Members_of_the_United_Kingdom_Parliament_for_English_constituencies', 'Indian_films', 'Year_of_death_missing', 'English_cricketers', 'Rivers_of_Romania', 'Main_Belt_asteroids', 'Asteroids_named_for_people', 'English-language_albums', 'English_television_actors', 'British_films', 'American_films', 'Fellows_of_the_Royal_Society', 'People_from_New_York_City', 'American_Jews', 'American_television_actors', 'American_film_actors', 'Debut_albums', 'Black-and-white_films', 'Year_of_birth_missing', 'Place_of_birth_missing_(living_people)', 'Article_Feedback_Pilot', 'American_military_personnel_of_World_War_II', 'Windows_games'])

In [13]:
len(categories.keys())

32

__Getting rid of those nodes in categories that are not in the reduced graph__

In [15]:
# We want only those nodes that are in the reduced graph.
graph_categories = {}
for key in categories.keys():
    article_set = set(categories.get(key)).intersection(nodes)
    graph_categories.update({key:article_set})

In [16]:
graph_categories_df = pd.DataFrame()
graph_categories_df['Categories'] = list(graph_categories.keys())
graph_categories_df['Articles'] = list(graph_categories.values())
art_len = [len(elem) for elem in list(graph_categories.values())]
graph_categories_df['Count'] = art_len

In [17]:
# after taking the intersection of each categories' value list with unique nodes in our reduced graph, we again check to 
# select those categories with higer value length of 3500
graph_categories_df = graph_categories_df[graph_categories_df['Count']>3500].reset_index(drop=True)

In [18]:
graph_categories_df

,Categories,Articles,Count
0,English_footballers,"{622642, 622644, 622647, 622649, 557114, 62266...",7538
1,The_Football_League_players,"{1671189, 622644, 622647, 622649, 557114, 1245...",7814
2,Association_football_forwards,"{1671189, 884758, 884785, 1671225, 884795, 884...",5097
3,Association_football_goalkeepers,"{737292, 737293, 1671183, 73743, 106515, 10651...",3737
4,Association_football_midfielders,"{1671168, 884755, 884774, 622637, 1671214, 167...",5827
5,Association_football_defenders,"{81920, 1359875, 1359879, 802827, 81936, 10651...",4588
6,Harvard_University_alumni,"{1638406, 1638409, 1245204, 294956, 1048629, 6...",5549
7,Major_League_Baseball_pitchers,"{1671480, 1278327, 1278328, 328097, 1278406, 3...",5192
8,Members_of_the_United_Kingdom_Parliament_for_E...,"{1048670, 1147049, 1147262, 1147295, 1507751, ...",6491
9,Indian_films,"{589824, 589825, 589826, 589827, 589828, 58982...",5568


__Doing the same cleaning for the dictionary__

In [19]:
# for having in our dictionary only categories with more than 3500 articles
for elem in list(graph_categories.keys()):
    if elem not in graph_categories_df['Categories'].tolist():
        del graph_categories[elem]

__all the noeds after cleaning the categories__

In [20]:
len(sorted({x for v in graph_categories.values() for x in v}))

139433

## Finding the shortest path

Given that the distance is the number of hops, and is optimal (shortest path.) We may keep track of visited nodes and current reachable nodes using Python's list/set. Starts from the first node and then keep hopping from the current set of nodes until we reach the target.
The point of visited-node list is to prevent visiting the visited nodes, resulting in a loop. And to get shortest distance, it's no use to make a revisit as it always makes the distance of resulting path longer.
This is a simple implementation of Breadth-first search. The efficiency partly depends on how to check visited nodes, and how to query adjacent nodes of given node. The Breadth-first search always guarantee to give optimal distance.

In [21]:
# Input category
C0_name = 'English_footballers'
C0 = graph_categories[C0_name][0:999]

# Finding the Shortest Path 

so i uploaded HM5-draft_Kat.ipynb, i changed only part 1, finally we received undirected graph. i checked it with Adjacency lists as for this check we have only two options Adjacency lists and Adjacency matrix(matrix is even more consuming, so it's the only way).

and i also changed formula of computing density of graph, as in Wiki formula with 2 in numerator, so i decided that its right.

also i wrote another code for computing average degree of nodes, but we don't need it as for undirected graphs we can use Samin's code.

In [25]:
def BFS_with_saving_path(graph, start):#this returns a tree for each node 
    visited = {start} #visited nodes
    queue = deque([(start, [])]) #putting the first node in a queue
    sav_path = {} #saved path will be a dictionary or a set?
    
    while queue: #until our queue is filled with something
        current, path = queue.popleft() #pop out in FIFO order, get the current node and the path(is only one node here)
        try:
            for neighbor in graph[current]: #for every neighbor of current node do as follows
                if neighbor not in visited: #if neighbor is not visited yet
                    queue.append((neighbor, path + [current]))
                    visited.add(neighbor)
                    sav_path[neighbor] = current
        except KeyError:
            pass
    return sav_path

In [26]:
def ShortestPath(graph, start, goal, tree):
    path = []
    if start == goal:
        return(len(path))
    elif goal not in tree:
        return(math.inf)
    else:
        current = goal
        while current != start:
            path.append(tree[current])
            current = tree[current]
        return(len(path))

In [27]:
def cal_(C0, key):
    Ci = graph_categories[key]#to calculate shortest paths for pairs in (C0, Ci)
    path_C0_Ci = []
    for start in tqdm(C0):#source node in C0
        tree = BFS_with_saving_path(out_Link, start)#gives us a tree from start in C0 and all the other nodes in the graph
        infinite = 0#to keep the number of paths that do not exist
        for end in Ci:#destination node in category Ci
            shortest_path_value = ShortestPath(out_Link, start, end, tree)
            if shortest_path_value == math.inf:
                #path_Ci_Cj_dict['inf'] = [path_Ci_Cj_dict['inf'][0] + 1]
                infinite += 1
            path_C0_Ci.append(shortest_path_value) # [5,2,4,3]   
    path_C0_Ci.append(infinite)
    return(path_C0_Ci)

In [ ]:
ranks = []
Ci_to_Cj_sp_values = []
for key in graph_categories.keys():
    if key != C0_name:
        Ci_to_Cj_sp_values = cal_(C0, key) # a list of values each for the shortest paths between nodes in C0 and Ci
    #finding the median
    ranks.append(tuple((key, np.median(Ci_to_Cj_sp_values))))  

 72%|███████████████████████████████████████████████████████                     | 5459/7538 [1:30:20<34:17,  1.01it/s]

In [80]:
start_time = time.time()
path_Ci_Cj_dict = {'inf':[0],'val':[]}#a dictionary with two keys, one 'inf' for counting number of infinites, the other, idk?!
distances_from_C0 = {'C1':[0,0,0]}#why do we have 3 zeros here?
#For category in categories:
for start in tqdm(C0):
    tree = BFS_with_saving_path(out_Link, start) #here we create a tree. Root is one node in C0
    for end in C1:
        #print(start)
        path = ShortestPath(out_Link,start,end,tree)
        #print(start, end, path)
        if path == math.inf:
            path_Ci_Cj_dict['inf'] = [path_Ci_Cj_dict['inf'][0] + 1]
        else:
            path_Ci_Cj_dict['val'].append(path) # [5,2,4,3]
    
    #print('path_Ci_Cj_dict', path_Ci_Cj_dict)
    distances_from_C0['C1'] = [distances_from_C0['C1'][0] + path_Ci_Cj_dict['inf'][0], \
                               distances_from_C0['C1'][1] + len(path_Ci_Cj_dict['val']), \
                               distances_from_C0['C1'][2] + sum(path_Ci_Cj_dict['val'])]
    path_Ci_Cj_dict = {'inf':[0],'val':[]}
print("--- %s seconds ---" % (time.time() - start_time))

100%|████████████████████████████████████████████████████████████████████████████| 3737/3737 [1:17:38<00:00,  1.47s/it]


--- 4658.674535274506 seconds ---


In [224]:
distances_from_C0

{'C1': [8889460, 10158029, 56407254]}

# RQ2 - Part2

In [22]:
initial_gr = networkx.DiGraph(out_Link)

In [25]:
len(C0)

7538

In [ ]:
#creating an induced graph from input category C0
C0 =  list(graph_categories.values())[0]
C0_sub = initial_gr.subgraph(C0)
in_degree_dict = C0_sub.in_degree()
#sort in_degree_dict here, based on the values
sorted_ = []
sorted_.append = sorted(in_degree_dict.items(), key=lambda x: x[1])#a list of tuples, sorted according to the 

In [ ]:
in_degree_dict

In [40]:
len([*in_degree_dict])

7538

__second_version__

In [ ]:

for category in tqdm(list(graph_categories.values())[1:]):
    empty_ = []# a list of lists. Each item in this list is a category cosists of articles that belong to it
    Ci = category #[23,45,..] a list of articles
    Ci = Ci.difference(C0) #we should not consider those nodes shared between C0 and Ci so we get the difference. difference is those nodes that belong to Ci and not C0
    induced_gr = initial_gr.subgraph(set(C0).union(set(Ci))# we create an induced sub_graph. 
    
    for i, node in enumertae(Ci):
        score = 0
        if len(induced_gr.in_edges(node)) == 0:
            score = 0
    
        else:
            for item in induced_gr.in_edges(node):
                if item[0] in Ci:
                    score += 1
                if item[0] in [*in_degree_dict]:
                    score += in_degree_dict[item[0]]
                
        empty_.append((node, score)) # in_degree_dict is not working for me here
    #sort in_degree_dict for all the nodes
    sorted_.append(sorted(empty_, key=lambda x: x[1]))  #a list of lists of sorted categories                                
                                     
    C0 = induced_gr.nodes()